# CMSC 422 Project 2: Unsupervised Learning

In this project, we will explore dimensionality reduction (PCA) and
clustering.

Files you'll edit:

    dr.py           Implementation of PCA
    clustering.py   Implementation of K-means (and variants)
    cifar10.py      Utilities to work with CIFAR10 data

Files you might want to look at:

    datasets.py     Some simple toy data sets
    digits          Digits data
    util.py         Utility functions, plotting, etc.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datasets
import util
import dr_soln as dr
import cifar10_soln as cifar10
import clustering_soln as clustering
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Part 0: PCA (40%)

Our first tasks are to implement PCA.  If implemented correctly, these
should be 5-line functions (plus the supporting code I've provided):
just be sure to use numpy's eigenvalue computation code.  **Implement
PCA in the function `pca` in `dr.py`.**

Our first test of PCA will be on Gaussian data with a known covariance
matrix.  First, let's generate some data and see how it looks, and see
what the *sample covariance* is:

In [ ]:
np.random.seed(422)

# Specify the sample covariance of the multivariate normal data
Si = util.sqrtm(np.array([[3,2],[2,4]]))
x = np.dot(np.random.randn(1000,2), Si)

plt.plot(x[:,0], x[:,1], 'b.')
np.dot(x.T,x) / np.real(x.shape[0])

(Note: The reason we have to do a matrix square-root on the covariance
is because Gaussians are transformed by standard deviations, not by
covariances.)

Note that the sample covariance of the data is almost exactly the true
covariance of the data.  If you run this with 100,000 data points
(instead of 1000), you should get something even closer to
`[[3,2],[2,4]]`.

Now, let's run PCA on this data.  We basically know what should
happen, but let's make sure it happens anyway.

In [ ]:
(P,Z,evals) = dr.pca(x, 2)
print('Z:', Z)
print('evals:', evals)


This tells us that the largest eigenvalue corresponds to the
direction `[ 0.57390327  0.8189231 ]` and the second largest corresponds to
the direction `[-0.8189231   0.57390327]`.  We can project the data onto
the first eigenvalue and plot it in red, and the second eigenvalue in
green.  (Unfortunately we have to do some ugly reshaping to get
dimensions to match up.)

In [ ]:
x0 = np.dot(np.dot(x, Z[:,0]).reshape(1000,1), Z[:,0].reshape(1,2))
x1 = np.dot(np.dot(x, Z[:,1]).reshape(1000,1), Z[:,1].reshape(1,2))
plt.plot(x[:,0], x[:,1], 'b.', x0[:,0], x0[:,1], 'r.', x1[:,0], x1[:,1], 'g.')

## PCA on a "real" example - CIFAR10

Now, let's look at some "real" data, we have preprocessed some data from [CIFAR](https://www.cs.toronto.edu/~kriz/cifar.html) for you. We have extracted 2000 16x16 images from the CIFAR10 dataset and have supplied the dataset to you in the form of a [pickled](https://docs.python.org/3.1/library/pickle.html) python dictionary whose keys correspond to the following:
    - 'X' : List with shape (2000, 784) corresponding to the 2000 CIFAR10 images
    - 'y' : The labels corresponding to images in X. The labels 0-9 corrspond to the categories, 
        `[airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck]`.

First, **implement `cifar10.load_data(...)`, so we can easily get load and use pickled data.** We can then take a look at some examples of the data. Fun!

In [ ]:
(X,y) = cifar10.load_data('data/cifar10.pkl')
cifar10.show_imgs(X[:64])

Let's apply PCA to our data and see what we find...

In [ ]:
(P,Z,evals) = dr.pca(X, X.shape[-1])

**Q1:** **Plot** the explained variance when you include up to $n$ eigenvectors for each possible value of $n$. How many eigenvectors do you have to include before you've accounted for 90% of the variance?  95%? (Hint: see function
`cumsum`.) 

In [ ]:
### Produce your plot here

**A1:** (Answer here)

Something interesting that we can do is to actually visualize the top eigenvectors. You will have to first implement `cifar10.normalize_img(...)`.

Let's plot the top 64 eigenvectors:

In [ ]:
print(Z.shape)
cifar10.show_imgs(Z.T[:64], normalize=True)

**Q2:** Why do we need to normalize the values in the eigenimages when we want to visualize them? What would happen if we did not?

**A2:**

**Q3:** Do these eigenimages look like images?  Should they?  Do these look like parts of images? Should they? Why or why not?

**A3:**

Lets take a closer look at two particular images...

In [ ]:
i = 10
n_pcs = 8
img = X[i]
pcs = P[i]
pc_order = np.argsort(-np.abs(pcs))[:n_pcs]
cifar10.show_img(img)
print("Principal components, sorted:", pc_order)
cifar10.show_imgs(Z.T[pc_order][:n_pcs], normalize=True)
print("*" * 20)
i = 55
img = X[i]
pcs = P[i]
pc_order = np.argsort(-np.abs(pcs))[:n_pcs]
cifar10.show_img(img)
print("Principal components, sorted:", pc_order)
cifar10.show_imgs(Z.T[pc_order][:10], labels=pc_order[:], normalize=True)

**Q4:** Look at the image above, look carefully at the principal components 0, 1, and 2. How do these relate to the above two images? Why do these principal components make sense (hint: say some thing about how most of the images in the dataset are composed/look like)?

**A4:**

# Part 1: K-means clustering (40%)

Your second task is to implement the largest distance heuristic for
kmeans clustering in `clustering.py`.

Note that we have already implemented the first initialization heuristic, the *deterministic* heuristic, where the initial K means are simply the first K data points the algorithm recieves. You will have to **implement `clustering.kmeans(...)`**.

Then we can quickly run through some basic experiments k-means:

**Hint:** While running, this will plot the results.  If you want
to turn that off, comment out the obvious line in the `kmeans`
function.  Plus, when it says "Press enter to continue", if you type
"q" and press enter, it will stop bugging you.

In [ ]:
mu0 = clustering.initialize_clusters(datasets.X2d, 2, 'determ')
(mu,z,obj) = clustering.kmeans(datasets.X2d, mu0)
print('mu', mu)
print('z', z)
print('obj', obj)

You can also play with another example:

In [ ]:
mu0 = clustering.initialize_clusters(datasets.X2d2, 4, 'determ')
(mu,z,obj) = clustering.kmeans(datasets.X2d2, mu0)

Now **implement the furthest first heuristic (`ffh`)**, and we can use this heuristic with K-means to cluster the CIFAR10 data.

In [ ]:
mu0 = clustering.initialize_clusters(X, 15, 'ffh')
(mu,z,obj) = clustering.kmeans(X, mu0, doPlot=False, verbose=False)

**Q5:** **Make a plot** of the K-means objective against iteration number. Why does the plot have a long tail? Why do we expect the plot to have this shape?

In [ ]:
# Make your plot here
### TODO: YOUR CODE HERE

**A5:**

Lets see if we can visualize the clusters...

In [ ]:
util.plotDatasetClusters(X[:,:2], mu[:, :2], z)

**Q6:** Why don't we see clear clusters here? what did we do wrong?

**A7:**

Lets do something better then!

In [ ]:
util.plotDatasetClusters(P[:,:2], mu[:, :2], z)

**Q7:** Is this plot better? Why does plotting the images using the first two PCs make sense? Qualitatively, what can we say about the clusters?

(Note: If you are having a hard time seeing the colors/clusters, please ask the instructors for more assistance.)

**A7:**

# Evaluating K-means (20%)

**Now implement the `km++` (K Means++), and `random` heuristics**

**Q8:** **Run kmeans 5 times each** with 15 centroids for the kmeans++, furthest first, and random selection, heuristics. For each heuristic, **answer and do** the following.
- For each heuristic, plot **one** plot of iteration against objective. (Each plot should have 5 lines).
- What was the average number of iterations it took for k-means to converge
- Which was the best heuristic?

In [ ]:
### TODO: YOUR CODE FOR Q7 HERE. Insert cells for different runs and answers as appropriate

**A8:**

**Q9:** Out of all your runs, select your "best" run (the run with the lowest objective) and, for the clustering from the best run, report the following:
- For each cluster:
    - 1) Report the modal class
    - 2) Report the percentage of the cluster that is the modal class
    - 3) Display 20 (a 2x10 grid) of sample points from the cluster
    - 4) Display the cluster mean
    - 5) Describe the cluster using one or two sentences.
- **Hint:**, for each fold, your output for 1-4) might look like:
<img src="kmeans_example_analysis.PNG" width="400">

In [ ]:
### YOUR CODE HERE FOR Q8

**A9:**

**Q10:**
- How successful was the clustering? Write a paragraph (5-7 sentences) describing how 
- Look at your sample images from your clusters and your cluster centroids, you should see a cluster whose modal label is airplane or boat (look for the cluster mean that vaguely looks like an object in the sky/ocean). Is this a good cluster? Why is it a good cluster? Why are both images of planes and boats in this cluster; how did kmeans get confused? What did kmeans "learn" (what features of the image did the algorithm pick up on)? Why might confusing images of boats and planes be problematic?

**A10:**

# Extra Credit! - "Really" evaluating K-means...

The CIFAR10 dataset that we have provided you is nice to play with because we have ground truth labels. However, in the real world, we don't use unsupervised learning on labeled datasets. For extra credit, try the some of the following:

- Without CIFAR10 labels, can you try to find out how many clusters or classes of images there really are? Try K-means clustering with different Ks and evaluate your result by implementing a metric like [silhuotte width](https://en.wikipedia.org/wiki/Silhouette_(clustering)). What patterns do you find, are there "subclasses" of classes in the dataset? Are there other patterns in the dataset other than just the 10 classes? Are the clusters any good? Can you use other featurizations or dimention reduction algorithms (like PCA) to improve your clustering? Why do they work?

- Or, find or generate an unlabeled dataset, perform dimention reduction and clustering and evaluate your findings.

Just show us that you have learned something new and attempt to apply the new things you have learned! Have fun.